# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east ballina,-28.8667,153.5833,21.00,57,0,1.71,AU,1717645848
1,1,vieux-habitants,16.0589,-61.7659,26.76,76,100,6.09,GP,1717645848
2,2,ubari,26.5921,12.7805,27.90,39,12,6.43,LY,1717645848
3,3,wailua homesteads,22.0669,-159.3780,29.24,81,40,4.63,US,1717645848
4,4,sitka,57.0531,-135.3300,9.94,76,100,3.60,US,1717645848


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='Humidity', 
    alpha=0.6, 
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
best_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
    (city_data_df["Cloudiness"] == 0) &
    (city_data_df["Wind Speed"] < 4.5)
]

# Drop any rows with null values
best_cities_df = best_cities_df.dropna()

# Display sample data
best_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,enoch,37.7733,-113.0244,26.14,19,0,3.09,US,1717645852
62,62,sirjan,29.4520,55.6814,26.81,12,0,4.07,IR,1717645853
221,221,san matias,-16.3667,-58.4000,21.26,46,0,2.20,BO,1717645868
289,289,novyy urgal,51.0736,132.5642,22.72,34,0,4.02,RU,1717645874
368,368,mazidagi,37.4775,40.4881,26.50,35,0,3.14,TR,1717645885


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
44,enoch,US,37.7733,-113.0244,19,
62,sirjan,IR,29.4520,55.6814,12,
221,san matias,BO,-16.3667,-58.4000,46,
289,novyy urgal,RU,51.0736,132.5642,34,
368,mazidagi,TR,37.4775,40.4881,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel within 10,000 meters
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
    }
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
enoch - nearest hotel: Travelodge by Wyndham Cedar City
sirjan - nearest hotel: خانه معلم سیرجان
san matias - nearest hotel: Hotel San Jose
novyy urgal - nearest hotel: No hotel found
mazidagi - nearest hotel: No hotel found
tabalah - nearest hotel: No hotel found
melilla - nearest hotel: Rusadir
qobustan - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
white hall - nearest hotel: Redwood Inn & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
44,enoch,US,37.7733,-113.0244,19,Travelodge by Wyndham Cedar City
62,sirjan,IR,29.4520,55.6814,12,خانه معلم سیرجان
221,san matias,BO,-16.3667,-58.4000,46,Hotel San Jose
289,novyy urgal,RU,51.0736,132.5642,34,No hotel found
368,mazidagi,TR,37.4775,40.4881,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
vacationing = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='Humidity', 
    alpha=0.6, 
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']
)

# Display the map
vacationing

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)